<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/Nb_Google_Colab/S06_Bases_Estadisticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <tr>
     <th><p><img alt="Colaboratory logo" height="80px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/83/Sena_Colombia_logo.svg/1045px-Sena_Colombia_logo.svg.png" align="left" hspace="10px" vspace="0px"></p></th> 
    <th><p><img alt="Colaboratory logo" height="80px" src="https://www.isa.co/wp-content/uploads/2020/11/logo.png" align="right" hspace="10px" vspace="0px"></p></th>
    <th><p><img alt="Colaboratory logo" height="80px" src="https://upload.wikimedia.org/wikipedia/commons/b/bf/EAFIT-2015.png" align="left" hspace="10px" vspace="0px"></p></th> 
     <th><h1>  Bases de Estadística Básica Aplicada </h1></th>
  </tr>








## Regresión Lineal

El problema de regresión lineal en una dimensión tiene como objetivo ajustar una variable numérica y continua,$y$, a una variable independiente $x$, por medio de un modelo lineal

$$y = mx + b$$

en donde los parámetros corresponden a:

- $m$ pendiente de la recta.
- $b$ intercepto con el eje $y$.

In [ ]:
#@title Librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from numpy.random import randn
from numpy.random import seed

In [ ]:
x=10*np.random.rand(100)
e=10*(np.random.rand(100)-0.5)
y=3*x+5+e
plt.scatter(x,y)
plt.plot(np.arange(0,11),3*np.arange(0,11)+5,c='r')
plt.show()

La correlación entre ambos datos esta dada por:

In [ ]:
# Cálculo de la correlación de pearson.
corr, _ = pearsonr(x, y)
corr

La línea recta sobre los puntos representa la tendencia del ajuste. En la figura se puede observar que esta línea no se ajusta a cada uno de los puntos; en su lugar, marca una tendencia promedio, por lo que se puede obtener un error en el ajuste a través de la distancia promedio de los puntos a la línea de tendencia:

In [ ]:
plt.errorbar(x,y,yerr=np.c_[e,np.zeros_like(e)].T,fmt='none',c='m')
plt.scatter(x,y)
plt.plot(np.arange(0,11),3*np.arange(0,11)+5,c='r')
plt.show()

### Mínimos Cuadrados: Solución Analítica

Al rededor de 1800 Carl Friederich Gauss y Adrien-Marie Legendre encontraron la forma de realizar un ajuste lineal, al minimizar la suma de los errores cuadráticos (MSE):

$$\text{MSE} = \frac{1}{N}\sum_{i=1}^N(\hat{y}_i-y_i)^2$$

en donde 

- $\hat{y}_i$ corresponde al i-ésimo valor predicho por el modelo.
- $y_i$ corresponde al i-ésimo valor del conjunto de datos de la variable objetivo.

Se puede mostrar que los valores de la pendiente $m$ y el intercepto $c$ que minimizan la ecuación $y = mx + b$ están dados por

$$m=\frac{\bar{xy}-\bar{x}\bar{y}}{\bar{x^2}-\bar{x}^2}$$

$$b=\bar{y}-m\bar{x}$$

en donde las cantidades con una barra corresponden a sus valores promedio.

Los valores optimos serán entonces

In [ ]:
N=len(x)
m=(np.sum(x*y)-np.sum(x)*np.sum(y)/N)/(np.sum(x*x)-np.sum(x)**2/N)
b=(np.sum(y)-m*np.sum(x))/N

print('m',m)
print('b',b)

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), m*np.arange(0,10,10/100)+b, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
 print(f"MSE: {mean_squared_error(y, m*x+b)}")

El problema multidimensional de un ajuste lineal puede ser escrito como:

$$y=a_0+a_1 x_1+a_2 x_2+...+a_m x_m=\sum_{j=0}^m a_m x_m$$

El cual puede ser resuelto analíticamente como:

$$\vec{a}=(a_0,a_1,...,a_m)=(X^T X)^{-1}X^T y$$

Para lo que ser requiere de la inversión de la matriz $X^T X$

In [ ]:
X=np.c_[np.ones_like(x),x]
params = np.matmul(np.linalg.inv(np.matmul(X.T,X)),np.matmul(X.T,y))
params

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), params[1]*np.arange(0,10,10/100)+params[0], c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, params[1]*x+params[0])}")

No obstante, si se poseen variabes características, la ecuación normal puede resultar computacionalmente lenta, debido a la inversión matricial que se reaquiere calcular. Para evitar esto, se hace uso de la técnica de descenso del gradiente.

##  Solución en `sklearn`:

La solución estándar en `sklearn` consiste en la siguientes líneas:

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(x.reshape(-1,1),y)
print(f"{lr.intercept_, lr.coef_[0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), lr.coef_[0]*np.arange(0,10,10/100)+lr.intercept_, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, lr.coef_[0]*x+lr.intercept_)}")

Si usamos decenso de gradiente, usamos el objeto de `sklearn` `SGDRegresor`:

In [ ]:
model = SGDRegressor(max_iter=1000, tol=0.001, eta0=0.1)
model.fit(x.reshape(-1,1),y)
print(f"{model.intercept_, model.coef_[0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), model.coef_[0]*np.arange(0,10,10/100)+model.intercept_, c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, model.coef_[0]*x+model.intercept_)}")

## Solución en `keras`:

En `keras`, para implementar una regresión lineal, se programa una sola capa sin función de activación:

In [ ]:
from tensorflow import keras

In [ ]:
keras.backend.clear_session()
model = keras.Sequential()
model.add(keras.layers.Dense(1, input_shape=(1,)))
model.summary()

In [ ]:
model.compile(keras.optimizers.Adam(learning_rate=0.9), 'mean_squared_error')

In [ ]:
model.fit(x,y,epochs=30)

In [ ]:
m,b = model.get_weights()
print(f"{b[0],m[0][0]}")

In [ ]:
plt.scatter(x,y)
plt.plot(np.arange(0,10,10/100), m[0][0]*np.arange(0,10,10/100)+b[0], c='r')
plt.legend(["Modelo","Datos"])
plt.show()

In [ ]:
print(f"MSE: {mean_squared_error(y, m[0][0]*x+b[0])}")

## Ejercicios

Para el siguiente conjunto de puntos

In [ ]:
#@title Conjunto de datos xE, yE
from sklearn.datasets import make_regression

xE,yE = make_regression(n_samples=200, n_features=1, noise=3)
plt.scatter(xE,yE)
plt.xlabel("xE")
plt.ylabel("yE")
plt.show()

realice los ejercicios enumerados a continuación.

### Ejercicio 1:

Para este ejercicio: 
1. Realice un ajuste lineal, implementando una función para el método de mínimos cuadrados
2. Desarrolle una función que grafique el conjunto de datos y el modelo. 
3. Calcule el valor del error cuadrático medio.

### Ejercicio 2:

Para este ejercicio:
1. Realice un ajuste lineal siguiendo el método estándard de `skelearn`. 
2. Haga uso de la función implementada en el ejercicio 1 para visualizar los datos y el modelo.
3. Calcule el valor del error cuadrático medio.

### Ejercicio 3:

Para este ejercicio:
1. Realice un ajuste lineal siguiendo el método de descenso de gradiente estocástico implementado en `sklearn`, usando varios modelos en donde evalue varios valores de los parámetros vistos:

  - `max_iter`
  - `tol`
  - `eta0`

2. Visualice los datos junto al modelo, en cada conjunto de parámetros analizado.
3. Calcule el valor del error cuadrático medio.

### Ejercicio 4:

Para este ejercicio:
1. Realice un ajuste lineal implementando el método de `keras`. 
2. Visualice el modelo junto a los datos.
3. Calcule el valor del error cuadrático medio.

<p><a name="redela"></a></p>

# 1. Redes elásticas

[[Contenidos]](#contents)

Como vimos anteriormente, podemos generar penalizaciones a las regresiones lineales haciendo cambios en su metrica de error.

Para la regresión Ridge se penaliza con: $\alpha \sum w_i^2$ (penalidad $L_2$), mientras que para Lasso se tiene que: 
$\alpha \sum |w_i|$ (penalidad $L_1$). Cada una de ellas tenía sus pro y sus contra. Pero es posible hacer una combinación de ambos metodos.

A las regresiones que usan una combinación de ambas penalidades se les conoce como **ElasticNet** (Redes elásticas) y definimos su error como:
$$\sum (Y_i- \hat Y_i)^2+\alpha \rho \sum |w_i| + \frac{\alpha(1-\rho)}{2}\sum w_i^2. $$

Note que cuando $ \rho=1$ tenemos la regresión Lasso, y con $\rho=0$ tendremos la de Ridge, por tanto en las redes elásticas $0\leq \rho\leq1$.

Hemos dado un paso más en la complejización del modelo ya que ahora debemos preocuparnos por el ajuste de 2 hiperparámetros para seleccionar el mejor modelo.

Para usar las redes elásticas en sklearn debemos importar la función 'ElasticNet' del modulo de modelos lineales.

In [ ]:
from sklearn.linear_model import ElasticNet

En la implementación de sklearn tenemos los hiperparámetros 'alpha' y 'l1_ratio' ($\rho$ en nuestra ecuación), con ellos controlaremos el comportamiento del regresor.

Tenga en cuenta que para valores de *l1_ratio* $\leq0.01$ el algoritmo de sklearn no es estable si usamos el valor de $\alpha$ por defecto y se hace necesario que nosotros mísmos ajustemos el valor. 

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv')

df.dropna(inplace=True)

X = df['ENGINESIZE'].values.reshape(-1,1)
y = df['CO2EMISSIONS'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn import metrics

#seleccionamos los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

#entrenamos el modelo
elastic = ElasticNet(alpha=0.01,l1_ratio=1,normalize=True)
elastic.fit(X_train,y_train)
y_pred = elastic.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X_test, y_test,  color='black',label=r'datos test')
plt.scatter(X_train, y_train,  color='red',alpha=0.4,label=r'datos entrenamiento')
plt.plot(X_test, y_pred, color='blue',label=r'predicción')
plt.legend(loc='best')
plt.xlabel(r'Engine size')
plt.ylabel(r'Price')
plt.show()

In [ ]:
print('MAE: ', metrics.mean_absolute_error(y_test, y_pred))
print('MSE: ', metrics.mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', elastic.score(X_test,y_test))

De nuevo, los hiperparámetros debemos seleccionarlos con una busqueda para determinar una buena combinación.

#### Ejercicio:

1. En el dataset de autos (a1): elimine los datos faltantes, convierta las variables categoricas en variables dummies (a2), y separe el dataset en datos de entrenamiento y prueba.

2. Use dos ciclos 'for' para recorrer $\alpha$ en la lista [0.01,0.1,1,10,100,1000] y l1_ratio en [0.1,0.3,0.6,0.9,0.99,0.999], entrenando un modelo ElasticNet para las combinaciones de éstos y evaluandolo con RMSE. Guarde el valor de cada RMSE en un arreglo.

3. Haga un mapa de calor (a3) con los valores de $\alpha$ y l1_ratio del punto anterior y los valores de RMSE.


a1. 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/automobileEDA.csv'

a2. `pd.get_dummies()`

a3. `sns.heatmap(Matriz, xticklabels, yticklabels, annot=True)`

Hacer doble click **aquí** para ver la solución:

<!-- Respuesta:


#1
#eliminamos datos nulos
df.dropna(inplace=True)
#creamos las variables dummies
df_dummies = pd.get_dummies(df)
#separamos el dataset
X = df_dummies.drop('price',axis=1)
y = df_dummies['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

#2
#Creamos la lista de alphas, l1_ratios y el arreglo de RMESs
alphas =  [0.01,0.1,1,10,100,1000] 
l1s = [0.1,0.3,0.6,0.9,0.99,0.999]
RMSEs = []

#Recorremos los valores en los arreglos
for alpha in alphas:
  for l1 in l1s:
    elastic = ElasticNet(alpha=alpha,l1_ratio=l1,normalize=True)
    elastic.fit(X_train,y_train)
    y_pred = elastic.predict(X_test)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #el metodo append agrega un valor al arreglo  
    RMSEs.append(RMSE)
RMSEs = np.array(RMSEs)

#3
import seaborn as sns
#pasamos de un arreglo a una matriz para usarlo en el mapa de calor
RMSEs = RMSEs.reshape(len(alphas),len(l1s))
sns.heatmap(RMSEs, xticklabels=alphas, yticklabels=l1s,annot=True )

--->

<p><a name="met"></a></p>

# 2. Métricas

[[Contenidos]](#contents)

En la sesión anterior estudiamos un modelo de regresión simple, por medio del cual se hace una predicción calculando una suma ponderada de las características de entrada, más una constante llamada término de sesgo (también llamado intercepto).

$$y=w_0+w_1x_1+w_2x_2+...+w_nx_n$$

Primero necesitamos de una medida de qué tan bien (o mal) el modelo se ajusta a los datos de entrenamiento. Esta medida de evaluación (función de costo) es el error calculado entre la recta generada $\hat{y}$ (o el hiperplano) a los puntos reales. El entrenamiento del modelo será entonces encontrar los valores de $w_i$ que minimicen dicha función de costo. Entre las métricas más populares encontramos:

* Error medio absoluto (MAE)

$$MAE = \frac{1}{m}\sum_{i=1}^{m}|\hat{y}_i -y_i|$$

* Error cuadrático medio (MSE)

$$MSE=\frac{1}{m}\sum_{i=1}^{m}\left(\hat{y}_i -y_i\right)^2$$

* Raíz del error cuadrático medio (RMSE)

$$RMSE=\sqrt{\frac{1}{m}\sum_{i=1}^{m}\left(\hat{y}({\bf x})_i -y_i\right)^2}$$

Estas métricas las podemos obtener del módulo [metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) de sklearn. Apliquémoslas al modelo lineal simple estudiado en la sesión anterior:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

archivo = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/automobileEDA.csv'
df = pd.read_csv(archivo)

#separemos nuestros datos en características y etiquetas
X = df['engine-size'].values.reshape(-1,1)
y = df['price'].values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

#seleccionamos los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

#entrenamos el modelo
linear  = LinearRegression(normalize=True)
linear.fit(X_train,y_train)

Obtengamos ahora los datos predichos por el modelo y calculemos las métricas

In [ ]:
y_pred = linear.predict(X_test)

In [ ]:
plt.scatter(X_test, y_test,  color='black',label=r'datos test')
plt.scatter(X_train, y_train,  color='red',alpha=0.4,label=r'datos entrenamiento')
plt.plot(X_test, y_pred, color='blue',label=r'predicción')
plt.legend(loc='lower right')
plt.xlabel(r'Engine size')
plt.ylabel(r'Price')
plt.show()

In [ ]:
print('MAE: ', metrics.mean_absolute_error(y_test, y_pred))
print('MSE: ', metrics.mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Y calculemos el coeficiente de correlación $R^2$

In [ ]:
print('R2: ', metrics.r2_score(y_test, y_pred))

Comprobemos además si los errores se distribuyen según una distribución normal, lo que nos da una prueba de la validez de nuestro modelo. El siguiente se conoce como un *histograma de residuos*

In [ ]:
sns.distplot((y_test - y_pred), bins = 50)
plt.show()

Una vez que hemos entrenado el modelo de regresión lineal las predicciones se obtienen rápidamente. La complejidad computacional es lineal con respecto a la cantidad de instancias y características sobre las que desea hacer predicciones. En otras palabras, hacer predicciones con el doble de instancias (o el doble de características) tomará aproximadamente el doble de tiempo de computo. Existen diferentes formas de entrenar un modelo de regresión lineal, más adecuado para casos en los que hay una gran cantidad de características o demasiadas instancias de entrenamiento para que quepan en memoria.





<p><a name="regpol"></a></p>

# 3. Regresión polinómica

[[Contenidos]](#contents)

**Regresión Polinómica**

Hasta ahora nos hemos centrado en la creación de modelos lineales simples y multivariados en donde la relación entre la/las variable/variables predictora/predictoras y la variable blanco corrsponde a una relación lineal. 

No obstante, en algunas ocasiones las tendencias de los datos presentan un comportamiento curvo. En estos caso debemos buscar otro modelo para representar los datos, como es el método de **regresión polinómica**. Dentro de la regreción polinómica, tenemos varias tipos de regresiones dependiendo del grado del polinómio que usemos:

* Cuadrática: Si el polinomio que usamos es grado dos.
* Cúbica: Si el polinomio que usamos es grado tres.
* Cuártica: Si el polinomio que usamos es grado cuatro.
* etc

Podemos llamar a todos estos casos regresiones polinómicas, ya que la relación entre la variable independiente $x$ y la variable dependiente  $y$ se modela mediante un polinomio de grado n en la variable $x$:

\begin{equation}
\hat{y}=w_0x^0+w_1x^1+w_2x^2\cdots w_nx^x = \sum_{i=0}^nw_ix^i
\end{equation}

En donde los $w$'s representan los parámetros del ajuste o la regresión 

**¿Cómo podemos abordar una regresión polinómica?**

Existe un "truco" que nos permite convertir una regresión polinómica en una regresión lineal múltiple. Si definimos:

* $x_1=x$
* $x_2=x^2$
* $x_3=x^3$
* $\cdot$
* $\cdot$
* $\cdot$
* $x_n=x^n$

podemos tratar el problema como una regresión lineal múltiple de la forma

\begin{equation}
\hat{y}=w_0x_0+w_1x_1+w_2x_2\cdots w_nx_x = \sum_{i=0}^nw_ix_i
\end{equation}

Por ende, la regresión polinómica se considera un caso especial de la regresión lineal múltiple, de tal forma que se pueden usar los mismos mecanismos que una regresión lineal para resolver el problema del modelado de los datos.

**Base de datos**

A continuación, tomaremos un conjunto de datos correspondientes a la clasificaciones de consumo de combustibles específicas de los modelos de carros y las emisiones estimadas de dióxido de carbono de nuevos vehiculos ligeros para la venta al por menor en Canada. Para mayor información puede consultar el [link](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64). La base de datos contienen características como:

- **MODELYEAR** Año del vehiculo.
- **MAKE** Marca o fabricante
- **MODEL** Modelo del vehiculo.
- **VEHICLE CLASS** Clase del vehiculo
- **ENGINE SIZE** Tamaño del motor.
- **CYLINDERS** Número de cilindros
- **TRANSMISSION** Tipo de transmisión
- **FUEL CONSUMPTION in CITY(L/100 km)** Consumo en ciudad en litros por cada 100 km.
- **FUEL CONSUMPTION in HWY (L/100 km)** Consumo en autopista en litros por cada 100 km.
- **FUEL CONSUMPTION COMB (L/100 km)** Consumo combinado en litros por cada 100 km.
- **CO2 EMISSIONS (g/km)** Emisión en gramos por kilómetros.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#Lectura de los datos
df = pd.read_csv("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv")

# Inspección visual del dataset
df.head()

Escojamos algunas características que estén relacionadas con la emisión de dioxido de carbono. Veamos la matriz de dispersión

In [ ]:
import seaborn as sns
sns.set(style="ticks")

sns.pairplot(df)

Intentemos construir un modelo, mediante un ajuste polinomial, de la emisión de dióxido de carbono en función de la característica "FUELCONSUMPTION_COMB_MPG"

In [ ]:
# Escogemos nuestra variable predictora como Engine Size:
X=df['FUELCONSUMPTION_COMB_MPG'].values.reshape(-1,1)

# Escogemos nuestra variable objetivo como las emisiones de CO2:
y=df['CO2EMISSIONS'].values.reshape(-1,1)

**Train and test dataframe**

Creemos ahora los grupos de train y test con los cuales entrenaremos y probaremos el modelo, respectivamente. Recordemos que este paso podemos hacerlo mediante la función `train_test_split` de la librería `sklearn.model_selection`:

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=20)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

<h2 id="evaluation">Regresión polinómica de los datos:</h2>

Intentemos crear un modelo cuadrático de la emisión de dióxido de carbono en función del tamaño del motor:

$\hat{y} = w_o + w_1 x + w_2 x^2$

En donde x reprenta la variable independiente 'FUELCONSUMPTION_COMB_MPG'. Para resolver este problema usamos la siguiente función:


__PloynomialFeatures()__ es una función de la  librería Scikit-learn, la cual emplea nuevo conjunto de características del conjunto de características original. Es decir, se generará una matriz compuesta de todas las combinaciones polinomiales de las características de grado menor o igual especificado en `degree`. Por ejemplo, suponiendo que el conjunto inicial solo tienen una característica, _FUELCONSUMPTION_COMB_MPG_, entonces, si especificamos `degree=2`, se generarán tres caracteristicas dadas por `degree=0`, `degree=1` y `degree=2`: 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)
train_x_poly = poly.fit_transform(x_train)
train_x_poly

El método **fit_transform** tomo los valores de x y crea una lista, elevando los valores en potencias enteras desde 0 hasta 2.

$
\begin{bmatrix}
    x_1\\
    x_2\\
    \vdots\\
    x_n
\end{bmatrix}
$
$\longrightarrow$
$
\begin{bmatrix}
    [ 1 & x_1 & x_1^2]\\
    [ 1 & x_2 & x_2^2]\\
    \vdots & \vdots & \vdots\\
    [ 1 & x_n & x_n^2]
\end{bmatrix}
$

De lo anterior se puede notar que se tiene la apariencia de un análisis de regresión multiple, lo que corrobora que la regresión polinómica representa un caso especial de la regresión lineal.

A partir de este punto, podemos tratar nuestro problema como un caso de regresión lineal, de tal forma que podemos usar los mismos mecanismos que se emplean para resolver este tipo de problemas. Podemos usar, por ejemplo, la función  __LinearRegression()__ para solucionar nuestro problema:

In [ ]:
clf = LinearRegression()
train_y_ = clf.fit(train_x_poly, y_train)
# The coefficients
print ('Coefficients: ', clf.coef_)
print ('Intercept: ',clf.intercept_)

En la celda anterior, __Coefficient__ e __Intercept__ corresponden a los parámetros del ajuste de la linea curva. 

Teniendo en cuenta que estamos tratando con una regresión lineal múltiple con 3 parámetros, los cuales representan la intersección y los coeficientes del hiperplano, sklearn los calcula a partir del nuevo conjunto de características. Veamos como luce el ajuste realizado sobre los datos de "COEMISSION" vs "FUELCONSUMPTION_COMB_MPG":

In [ ]:
plt.scatter(X, y,  color='blue')
XX = np.arange(0.0, 60.0, 0.1)
yy = clf.intercept_[0]+ clf.coef_[0][1]*XX+ clf.coef_[0][2]*XX*XX
plt.plot(XX, yy, '-r')
plt.xlabel("FUELCONSUMPTION_COMB_MPG")
plt.ylabel("Emission")

<h2 id="evaluation">Evaluación</h2>

In [ ]:
#con la regresion lineal
linear = LinearRegression()
linear.fit(x_train,y_train)

y_pred = linear.predict(x_test)

print('MAE: %.3f '% metrics.mean_absolute_error(y_test, y_pred))
print('MSE: %.3f'% metrics.mean_squared_error(y_test, y_pred))
print('RMSE: %.3f'% np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: %.3f'% metrics.r2_score(y_test, y_pred))

In [ ]:
#con la regresion polinómica
from sklearn import metrics
test_x_poly = poly.fit_transform(x_test)
test_y_ = clf.predict(test_x_poly)

print('MAE: %.3f' % metrics.mean_absolute_error(test_y_, y_test))
print('MSE: %.3f' % metrics.mean_squared_error(test_y_, y_test))
print('RMSE: %.3f' % np.sqrt(metrics.mean_squared_error(test_y_, y_test)))
print('R2: %.3f'% metrics.r2_score(test_y_, y_test))

# Problema:

Relice un ajuste a un polinomio de grado tres en donde aplique cada uno de los pasos mostrados la regresión polinómica anterior, y evalue si obtiene un mejor modelo.

Hacer doble click <b>aquí</b> para ver la solución:

<!-- Respuesta:

# Código
poly3 = PolynomialFeatures(degree=3)
train_x_poly3 = poly3.fit_transform(x_train)

clf3 = LinearRegression()
train_y3_ = clf3.fit(train_x_poly3, y_train)

#coeficientes
print ('Coefficients: ', clf3.coef_)
print ('Intercept: ',clf3.intercept_)

#graficación
plt.scatter(X, y, color="b")
XX = np.arange(0.0, 60.0, 0.1)
Y = clf3.intercept_ + clf3.coef_[0][1]*XX + clf3.coef_[0][2]*XX**2 + clf3.coef_[0][3]*XX**3
plt.plot(XX,Y,"r-" )
plt.show()

test_x_poly3 = poly3.fit_transform(x_test)
test_y3_ = clf3.predict(test_x_poly3)

print('MAE: %.3f ' % metrics.mean_absolute_error(y_test, test_y3_))
print('MSE: %.3f' % metrics.mean_squared_error(y_test, test_y3_))
print('RMSE: %.3f' % np.sqrt(metrics.mean_squared_error(y_test, test_y3_)))
print('R2-score: %.3f'% metrics.r2_score(y_test, test_y3_))

--->

# Modelos no lineales

Como bien sabemos, si los datos no presentan una tendencia lineal entre las características y las variables objetivo, debemos buscar ajustes a funciones no lineales para la construcción de modelos. Veamos algunas funciones de uso comun para el ajuste de modelos y un ejemplo práctico del crecimiento del producto interno bruto chino.

Las regresiones no lineales representan una relación entre variabres independientes $x$'s y una variable dependiente $y$, lo que resulta en un modelado mediante una función no lieal de los datos. En principio, cualquier relación que no es lineal, puede representarce mediante un polinomio de grado $k$. Por ejemplo: 

$$ \ y = a x^3 + b x^2 + c x + d \ $$

Además, las funciones no lineales pueden tener elementos exponenciales, logarítmicos, fracciones, entre otros. Por 
ejemplo, una función de la forma:

$$ y = \log(a x^5 + b x^3 + c x + d)$$

## Función exponencial

Una función exponencial de base c, se define como 

$Y = a + b c^X$

en donde $b\neq0$, $c > 0$ , $c\neq1$, y la X es un número real. La base, $c$, es una constante y el exponente, $X$, es una variable. Un ejemplo gráfico de la función se presenta a continuación:

In [ ]:
X = np.arange(-5.0, 5.0, 0.1)

Y= np.exp(X)

plt.plot(X,Y) 
plt.ylabel('Variable dependiente')
plt.xlabel('Variable independiente')
plt.title('Función exponencial')
plt.show()

## Función logarítmica

La función logarítmica es la función inversa de la función exponecial, y se representa como:

\begin{equation}
y = \log(X)
\end{equation}

Una representación gráfica de esta función se muestra a continuación:

In [ ]:
X = np.arange(-5.0, 5.0, 0.1)

Y = np.log(X)

plt.plot(X,Y) 
plt.ylabel('Dependent Variable')
plt.xlabel('Indepdendent Variable')
plt.title('Función Logarítmica')
plt.show()

## Función sigmoide/logística

La función sigmoide tiene la forma 

$$ Y = a + \frac{b}{1+ c^{(X-d)}}$$

Y veremos, más adelante, en este curso, una aplicación en la la regresión logística. Una representación gráfica de la función se presenta a continuación:

In [ ]:
X = np.arange(-5.0, 5.0, 0.1)


Y = 1-4/(1+np.power(3, X-2))

plt.plot(X,Y) 
plt.ylabel('Dependent Variable')
plt.xlabel('Indepdendent Variable')
plt.show()

## Ejemplo de regresión no lineal:

Intentemos encontra un modelo no lineal para representar los datos del producto interno bruto de China, entre los años 1960 a 2014. El dataset se presenta a continuación:

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/china_gdp.csv")
df.head(10)

### Gráfica del dataset ###

Veamos como luce el dataset:

In [ ]:
plt.figure(figsize=(8,5))
x_data, y_data = (df["Year"].values, df["Value"].values)
plt.plot(x_data, y_data, 'ro')
plt.ylabel('GDP')
plt.xlabel('Year')
plt.show()

Esta gráfica parece seguir un comportamiente logístico o exponencial. El crecimiento lento del PIB empieza alrededor del año 1995, y a partir del año 2005 empieza a ser significativo, para luego caer muy poco alrededor del 2010. 

### Determinación del modelo ###

De una inspección inical, podemos determinar que una aproximación logística podría ser adecuada, ya que empieza a crecer lentamente, y aumenta a mitad de camino, para desacelerarse un poco al final. Veamos la siguiente función:

In [ ]:
X = np.arange(-5.0, 5.0, 0.1)
Y = 1.0 / (1.0 + np.exp(-X))

plt.plot(X,Y) 
plt.ylabel('Dependent Variable')
plt.xlabel('Indepdendent Variable')
plt.show()

De la ecuación de la función logística:

$$ \hat{Y} = \frac1{1+e^{\beta_1(X-\beta_2)}}$$

Tenemos que

$\beta_1$: contrala la inclinación de la curva,

$\beta_2$: proyecta la curva en x.

### Construcción del modelo###

Construyamos nuestro modelo de regresión e inicialicemos los parámetros:

In [ ]:
def sigmoid(x, Beta_1, Beta_2):
     y = 1 / (1 + np.exp(-Beta_1*(x-Beta_2)))
     return y

Veamos un ajuste preliminar "a  mano" de una función sigmoide a los datos

In [ ]:
beta_1 = 0.10
beta_2 = 2010.0

#logistica
Y_pred = sigmoid(x_data, beta_1 , beta_2)

#plot predicción inicial
plt.plot(x_data, Y_pred*15000000000000.)
plt.plot(x_data, y_data, 'ro')

El objetivo en este modelo es encontrar los parámetros que mejor ajustan la curva a los datos. Procedamos a normalizarlos:

In [ ]:
# normalicemos los datos
xdata =x_data/max(x_data)
ydata =y_data/max(y_data)

Usemos el método __curve_fit__ `scipy`, que usa un ajuste de mínimos cuadrados no lineal para ajustar nuesta función sigmoide a los datos. El algoritmo ajusta iterativamente los parámetros, de tal forma que la suma de los residuos cuadrados $sig(x_{data}, *popt) - y_{data}$ se minimiza.

In [ ]:
from scipy.optimize import curve_fit
popt, pcov = curve_fit(sigmoid, xdata, ydata)
#imprimamos los parámetros
print(" beta_1 = %f, beta_2 = %f" % (popt[0], popt[1]))

In [ ]:
# Visualización del modelo
x = np.linspace(1960, 2015, 55)
x = x/max(x)
plt.figure(figsize=(8,5))
y = sigmoid(x, *popt)
plt.plot(xdata, ydata, 'ro', label='data')
plt.plot(x,y, linewidth=3.0, label='fit')
plt.legend(loc='best')
plt.ylabel('GDP')
plt.xlabel('Year')
plt.show()

## Problema:
Evalue la precisión del modelo creado anteriormente.

Hacer doble click <b>aquí</b> para ver la solución:

<!-- Respuesta:

#Reshape data
X=xdata.reshape(-1,1)
y=ydata.reshape(-1,1)

#Split 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xdata,ydata, test_size=0.2, random_state=1)

# crear el modelo
popt, pcov = curve_fit(sigmoid, x_train, y_train)

# predicciones sobre el dataset de test
y_hat = sigmoid(x_test, *popt)

# Evaluacion
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat - y_test) ** 2))
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_hat , y_test) )
--->